In [1]:
import os, sys
sys.path.insert(0, '/clusterCAD')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "clusterCAD.settings")
import django
django.setup()
import pks.models
import compounddb.models

In [2]:
# save compounds in the database
caf = compounddb.models.Compound(name="Caffeine", smiles="O=C2N(c1ncn(c1C(=O)N2C)C)C")
# caf.computeInchi()
caf.save()

cw = compounddb.models.Compound(name="cw", smiles="CCCC1=CC=C(C=C1)C(C)C(=O)O")
# cw.computeInchi()
cw.save()

In [3]:
caf.inchiKey
# caf.delete()

'RYYVLZVUVIJVGH-UHFFFAOYSA-N'

In [4]:
# test substructure search
compounds = compounddb.models.Compound.objects.raw('select \"inchiKey\" from compounddb_compound where m@>\'CCCc1ccc(C(C)C(=O)O)cc1\';')
[x.name for x in compounds]

['cw']

In [5]:
# query for top 10 hits to make a desired compound
queryString = 'CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)C(O)C(C)C(=O)C(C)C(O)C1C'
results = compounddb.models.Compound.atomPairSearch(queryString, maxHits=10, minSim=0.2)
[(result[0], result[1].mol()) for result in results]

[(1.0, <rdkit.Chem.rdchem.Mol at 0x7f8101165868>),
 (0.683602771362587, <rdkit.Chem.rdchem.Mol at 0x7f81006dc048>),
 (0.59349593495935, <rdkit.Chem.rdchem.Mol at 0x7f81006dc0b0>),
 (0.551487414187643, <rdkit.Chem.rdchem.Mol at 0x7f81006dc118>),
 (0.538321167883212, <rdkit.Chem.rdchem.Mol at 0x7f81006dc180>),
 (0.534736842105263, <rdkit.Chem.rdchem.Mol at 0x7f81006dc1e8>),
 (0.518918918918919, <rdkit.Chem.rdchem.Mol at 0x7f81006dc250>),
 (0.51473136915078, <rdkit.Chem.rdchem.Mol at 0x7f81006dc2b8>),
 (0.508982035928144, <rdkit.Chem.rdchem.Mol at 0x7f81006dc320>),
 (0.491467576791809, <rdkit.Chem.rdchem.Mol at 0x7f81006dc388>)]

In [6]:
# find which gene clusters, subunit, and module each top hit is from
[[(module.subunit.cluster, module.subunit, module)
  for module in pks.models.Module.objects.filter(product=result[1])]
     for result in results]

for result in results:
    print(result[0])
    for module in pks.models.Module.objects.filter(product=result[1]):
        print((module.subunit.cluster, module.subunit, module))

1.0
(<Cluster: Erythromycin pks gene cluster>, <Subunit: eryAIII pks subunit>, <Module: 2>)
0.683602771362587
(<Cluster: Methymycin / pikromycin pks gene cluster>, <Subunit: pikAIV pks subunit>, <Module: 1>)
0.59349593495935
(<Cluster: Megalomicin pks gene cluster>, <Subunit: megAIII pks subunit>, <Module: 2>)
(<Cluster: Erythromycin pks gene cluster>, <Subunit: eryAIII pks subunit>, <Module: 2>)
0.551487414187643
(<Cluster: Erythromycin pks gene cluster>, <Subunit: eryAIII pks subunit>, <Module: 1>)
(<Cluster: Erythromycin pks gene cluster>, <Subunit: eryAIII pks subunit>, <Module: 1>)
(<Cluster: Megalomicin pks gene cluster>, <Subunit: megAIII pks subunit>, <Module: 1>)
0.538321167883212
(<Cluster: Lankamycin pks gene cluster>, <Subunit: lkmAIII pks subunit>, <Module: 2>)
0.534736842105263
(<Cluster: Lankamycin pks gene cluster>, <Subunit: lkmAIII pks subunit>, <Module: 1>)
0.518918918918919
(<Cluster: Soraphen pks gene cluster>, <Subunit: sorB pks subunit>, <Module: 5>)
0.5147313691